MIAS-breast

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)
    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]
    return right - left - 1

def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 20, 0, img)
    height, _ = img.shape
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]
    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]
    return img_copy[row_ind][:, col_ind]

# 定义路径
TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = '../classification_data/MIAS-breast'
SPLIT_CSV_PATH = '../classification_data/classification_split.csv'

# 读取数据划分CSV文件
split_df = pd.read_csv(SPLIT_CSV_PATH)
split_df = split_df[split_df['dataset'] == 'MIAS-breast']

# 读取TXT文件
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

# 跳过标题行
lines = lines[1:]

# 初始化一个字典，用于合并相同的 refnum 信息
merged_data = {}

for line in lines:
    parts = line.split()
    refnum = parts[0]
    bg = str(parts[1]).replace(' ', '')
    cls = str(parts[2]).replace(' ', '')
    severity = str(parts[3]).replace(' ', '') if len(parts) > 3 else 'N'
    if refnum in merged_data:
        merged_data[refnum]['class'].add(cls)
        if severity == 'M':
            merged_data[refnum]['severity'] = 'M'
    else:
        merged_data[refnum] = {
            'background': bg,
            'class': {cls},
            'severity': severity
        }

# 处理并保存数据
for refnum, info in merged_data.items():
    # 查找对应的data_split
    split_info = split_df[split_df['data_name'] == refnum]
    if split_info.empty:
        print(f"No split info found for {refnum}")
        continue
    
    data_split = split_info['data_split'].values[0]
    
    # 读取 PGM 文件
    pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
    if os.path.exists(pgm_file):
        img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
        img = ExtractBreast(img)
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        # 创建输出目录
        output_dir = os.path.join(OUTPUT_BASE_PATH, data_split, refnum)
        os.makedirs(output_dir, exist_ok=True)

        # 保存图像为 JPEG 格式
        jpg_output_path = os.path.join(output_dir, 'img.jpg')
        cv2.imwrite(jpg_output_path, img)

        # 保存标签为 NumPy 文件字典
        info_dict = {
            'Composition': info['background']
        }
        npy_path = os.path.join(output_dir, 'info_dict.npy')
        np.save(npy_path, info_dict)

        print(f"Processed {refnum} for {data_split} set")
    else:
        print(f"PGM file for {refnum} not found.")

print("Processing completed.")

Processed mdb001 for Train set
Processed mdb003 for Train set
Processed mdb005 for Train set
Processed mdb007 for Train set
Processed mdb009 for Train set
Processed mdb011 for Train set
Processed mdb014 for Train set
Processed mdb015 for Train set
Processed mdb018 for Train set
Processed mdb019 for Train set
Processed mdb020 for Train set
Processed mdb021 for Train set
Processed mdb022 for Train set
Processed mdb023 for Train set
Processed mdb024 for Train set
Processed mdb025 for Train set
Processed mdb027 for Train set
Processed mdb028 for Train set
Processed mdb029 for Train set
Processed mdb030 for Train set
Processed mdb031 for Train set
Processed mdb032 for Train set
Processed mdb033 for Train set
Processed mdb035 for Train set
Processed mdb036 for Train set
Processed mdb037 for Train set
Processed mdb038 for Train set
Processed mdb040 for Train set
Processed mdb041 for Train set
Processed mdb042 for Train set
Processed mdb044 for Train set
Processed mdb045 for Train set
Processe

cropped-classification

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def crop_and_save(img, x, y, radius, output_path):
    h, w = img.shape
    x = int(x)
    y = w - int(y)
    radius = int(radius)
    x1 = max(x - radius, 0)
    y1 = max(y - radius, 0)
    x2 = min(x + radius, img.shape[1])
    y2 = min(y + radius, img.shape[0])

    cropped_img = img[y1:y2, x1:x2]
    cropped_img = cv2.normalize(cropped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    cv2.imwrite(output_path, cropped_img)


TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = '../classification_data/MIAS-finding'
SPLIT_CSV_PATH = '../classification_data/classification_split.csv'


split_df = pd.read_csv(SPLIT_CSV_PATH)
split_df = split_df[split_df['dataset'] == 'MIAS-finding']


with open(TXT_PATH, 'r') as file:
    lines = file.readlines()


lines = lines[1:]
lines = [line for line in lines if len(line.split()) > 4]


refnum_counter = {}
for line in lines:
    parts = line.split()
    refnum = parts[0]
    cls = str(parts[2]).replace(' ', '')
    severity = str(parts[3]).replace(' ', '')
    x, y, radius = parts[4], parts[5], parts[6]


    if refnum in refnum_counter:
        refnum_counter[refnum] += 1
    else:
        refnum_counter[refnum] = 1
    refnum_with_suffix = f"{refnum}_{refnum_counter[refnum]}"


    split_info = split_df[split_df['data_name'] == refnum_with_suffix]
    if split_info.empty:
        print(f"No split info found for {refnum_with_suffix}")
        continue
    
    data_split = split_info['data_split'].values[0]


    pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
    if os.path.exists(pgm_file):
        img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
        output_dir = os.path.join(OUTPUT_BASE_PATH, data_split, refnum_with_suffix)
        os.makedirs(output_dir, exist_ok=True)

        jpg_output_path = os.path.join(output_dir, 'img.jpg')
        crop_and_save(img, x, y, radius, jpg_output_path)


        info_dict = {
            'Finding': [cls],
            'Pathology': severity
        }
        

        updated_categories = []
        for category in info_dict['Finding']:
            if category == 'CALC':
                updated_categories.append('Calcification')
            elif category == 'CIRC':
                updated_categories.append('Circumscribed masses')
            elif category == 'SPIC':
                updated_categories.append('Spiculated masses')
            elif category == 'ARCH':
                updated_categories.append('Architectural distortion')
            elif category == 'ASYM':
                updated_categories.append('Asymmetry')
            elif category == 'MISC':
                updated_categories.append('Miscellaneous')
        if updated_categories:
            info_dict['Finding'] = updated_categories
        

        if 'Pathology' in info_dict:
            if info_dict['Pathology'] == 'B':
                info_dict['Pathology'] = 'Benign'
            elif info_dict['Pathology'] == 'M':
                info_dict['Pathology'] = 'Malignant'
        
        npy_path = os.path.join(output_dir, 'info_dict.npy')
        np.save(npy_path, info_dict)

        print(f"Processed {refnum_with_suffix} for {data_split} set")
    else:
        print(f"PGM file for {refnum} not found.")

print("Processing completed.")

Processed mdb219 for Train set
Processed mdb013 for Train set
Processed mdb170 for Train set
Processed mdb256 for Train set
Processed mdb097 for Train set
Processed mdb178 for Train set
Processed mdb091 for Train set
Processed mdb105 for Train set
Processed mdb107 for Train set
Processed mdb083 for Train set
Processed mdb248 for Train set
Processed mdb214 for Train set
Processed mdb271 for Train set
Processed mdb102 for Train set
Processed mdb063 for Train set
Processed mdb241 for Train set
Processed mdb015 for Train set
Processed mdb267 for Train set
Processed mdb152 for Train set
Processed mdb030 for Train set
Processed mdb090 for Train set
Processed mdb252 for Train set
Processed mdb239 for Train set
Processed mdb095 for Train set
Processed mdb274 for Train set
Processed mdb080 for Train set
Processed mdb204 for Train set
Processed mdb081 for Train set
Processed mdb264 for Train set
Processed mdb058 for Train set
Processed mdb134 for Train set
Processed mdb005 for Train set
Processe